In [1]:
# encoding: utf-8
import pandas as pd
import numpy as np
#!pip install mlxtend

from sklearn import model_selection

from sklearn import tree #C4.5
from sklearn.ensemble import BaggingClassifier #Bagging
from sklearn.ensemble import AdaBoostClassifier #Boosting
from sklearn.ensemble import RandomForestClassifier #Random Forests
from sklearn.ensemble import GradientBoostingClassifier #XGBoost
from mlxtend.classifier import StackingClassifier #Stacking

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import accuracy_score
import time
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler, StandardScaler

pd.options.display.max_colwidth = 150

data_glass = pd.read_csv("glass.data")

# to store the models and their results (to report)
models, results, tempos = list(), list(), list()

print('Glass:', np.shape(data_glass))

('Glass:', (214, 11))


In [2]:
data_glass = shuffle(data_glass)
data_glass.head()

,a,b,c,d,e,f,g,h,i,j,target
195,196,1.51545,14.14,0.00,2.68,73.39,0.08,9.07,0.61,0.05,7
38,39,1.52213,14.21,3.82,0.47,71.77,0.11,9.57,0.00,0.00,1
29,30,1.51784,13.08,3.49,1.28,72.86,0.60,8.49,0.00,0.00,1
41,42,1.51755,12.71,3.42,1.20,73.20,0.59,8.64,0.00,0.00,1
58,59,1.51754,13.48,3.74,1.17,72.99,0.59,8.03,0.00,0.00,1


In [3]:
y = data_glass['target']
data_glass = data_glass.drop(columns = ['target', 'a'])

In [4]:
# Normalizing the Glass dataset: standardize features by removing the mean and scaling to unit variance
scaler = StandardScaler()
scaler.fit(data_glass)
data_glass[:] = scaler.transform(data_glass)
data_glass.head()

,b,c,d,e,f,g,h,i,j
195,-0.962261,0.898681,-1.865511,2.479601,0.956429,-0.640968,0.079614,0.876823,-0.072105
38,1.242534,0.984603,0.789048,-1.957244,-1.140023,-0.594861,0.431770,-0.352877,-0.586451
29,-0.173420,-0.402422,0.559728,-0.331070,0.270553,0.158213,-0.328888,-0.352877,-0.586451
41,-0.269137,-0.856581,0.511084,-0.491680,0.710549,0.142844,-0.223241,-0.352877,-0.586451
58,-0.272438,0.088560,0.733456,-0.551908,0.438787,0.142844,-0.652871,-0.352877,-0.586451


In [5]:
C45 = tree.DecisionTreeClassifier() 

clf1 = KNeighborsClassifier(n_neighbors=1)
clf2 = GaussianNB()
lr = LogisticRegression()

all_models=[C45, 
            BaggingClassifier(C45, max_samples=0.4),
            AdaBoostClassifier(C45, n_estimators=100),
            RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0),
            GradientBoostingClassifier(learning_rate=0.1, n_estimators=100, subsample=1.0),
            clf1,
            clf2,
            StackingClassifier(classifiers=[clf1, clf2, C45], meta_classifier=lr)]

models.extend(all_models)

In [6]:
kf = KFold(n_splits=10, shuffle=True, random_state=2019)    

Running models for Glass dataset:

In [7]:
#For each model:
for model in all_models:
    print('\n')
    print('======= Model: ', model, '=======')
    # for each fold:
    accuracies = list()
    times = list()
    
    for i, (train_index, test_index) in enumerate(kf.split(data_glass)):
        #Builds the train and validation dataset, according to the current fold:
        y_train, y_valid = y.iloc[train_index].copy(), y.iloc[test_index]
        X_train, X_valid = data_glass.iloc[train_index,:].copy(), data_glass.iloc[test_index,:].copy()
        start = time.time()
        model.fit(X_train, y_train)
        end = time.time()
        times.append(end - start)
        pred = model.predict(X_valid)
        # print(confusion_matrix(y_valid, pred, labels=[1,2,3,5,6,7]))
        acc = accuracy_score(pred, y_valid)
        accuracies.append(acc) 
        
    print('Final results:')
    print('Mean accuracy:', np.mean(accuracies))      
    print('Mean traning model time: ', np.mean(times))
    results.append(np.mean(accuracies))
    tempos.append(np.mean(times))
    




('======= Model: ', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), '=======')
Final results:
('Mean accuracy:', 0.6683982683982683)
('Mean traning model time: ', 0.0028618812561035157)


('======= Model: ', BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=0.4, n_estima

/home/afonso/.local/lib/python2.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/afonso/.local/lib/python2.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Final results:
('Mean accuracy:', 0.6214285714285714)
('Mean traning model time: ', 0.019553089141845705)


In [8]:
for model in all_models:
    print('\n')
    print('======= Model: ', model, '=======' + "\n\n")



('======= Model: ', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), '=======\n\n')


('======= Model: ', BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=0.4, n_estimators=10, n_jobs=None, oob_score=False,
         random_state=None, verbose=0, warm_start=False), '=====

In [9]:
data = {'Accuracy': results, 'Time': tempos, 'Model': models}
dataframe = pd.DataFrame(data=data)
dataframe = dataframe.sort_values(by=['Accuracy'], ascending=False)
dataframe

,Accuracy,Model,Time
4,0.767316,"([DecisionTreeRegressor(criterion='friedman_mse', max_depth=3,\n max_features=None, max_leaf_nodes=None,\n min_impurity_decrea...",0.443793
1,0.734848,"(DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,\n max_features=None, max_leaf_nodes=None,\n min...",0.021968
5,0.701515,"KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',\n metric_params=None, n_jobs=None, n_neighbors=1, p=2,\n ...",0.001301
2,0.682035,"(DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,\n max_features=None, max_leaf_nodes=None,\n min...",0.003909
0,0.668398,"DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,\n max_features=None, max_leaf_nodes=None,\n min_...",0.002862
3,0.622294,"(DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,\n max_features='auto', max_leaf_nodes=None,\n min_...",0.247791
7,0.621429,"StackingClassifier(average_probas=False,\n classifiers=[KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',\n ...",0.019553
6,0.448052,"GaussianNB(priors=None, var_smoothing=1e-09)",0.001680


In [10]:
dataframe.to_csv('glass_results.csv',index=False)